In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
import datetime
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [2]:
data = pd.read_csv("phase2_train_v0.csv")
data['image_path'] = "./phase2_train_v0/final/"+data['img_fName']
data

,img_fName,img_w,img_h,bbx_xtl,bbx_ytl,bbx_xbr,bbx_ybr,class_label,image_path
0,train_00000.jpeg,2448,3264,1301,1546,1641,2096,albopictus,./phase2_train_v0/final/train_00000.jpeg
1,train_00001.jpeg,3024,4032,900,1897,1950,2990,albopictus,./phase2_train_v0/final/train_00001.jpeg
2,train_00002.jpeg,768,1024,220,58,659,808,albopictus,./phase2_train_v0/final/train_00002.jpeg
3,train_00003.jpeg,3456,4608,1169,2364,1586,2826,albopictus,./phase2_train_v0/final/train_00003.jpeg
4,train_00004.jpeg,1024,1365,129,231,697,1007,culex,./phase2_train_v0/final/train_00004.jpeg
...,...,...,...,...,...,...,...,...,...
10352,train_10352.jpeg,2064,1376,0,139,1344,1253,albopictus,./phase2_train_v0/final/train_10352.jpeg
10353,train_10353.jpeg,2664,3996,821,1481,1564,2706,albopictus,./phase2_train_v0/final/train_10353.jpeg
10354,train_10354.jpeg,1157,1157,367,315,676,764,albopictus,./phase2_train_v0/final/train_10354.jpeg
10355,train_10355.jpeg,3000,4000,1064,2463,1442,2917,albopictus,./phase2_train_v0/final/train_10355.jpeg


In [3]:
# Configurar ImageDataGenerator con aumento de datos para las clases minoritarias
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
# Crear generador de entrenamiento
train_generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col='image_path',
    y_col='class_label',
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=32,
    subset='training'
)

Found 8286 validated image filenames belonging to 6 classes.


In [5]:
# Crear generador de validación
validation_generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col='image_path',
    y_col='class_label',
    target_size=(128, 128),
    class_mode='categorical',
    batch_size=32,
    subset='validation'
)

Found 2071 validated image filenames belonging to 6 classes.


In [6]:
# Definir el modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 6 clases de especies de mosquitos
])

C:\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
# Configurar el callback de TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
# Crear el callback para reducir la tasa de aprendizaje cuando la validación no mejore
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [10]:
# Entrenar el modelo con el callback de reducción de tasa de aprendizaje
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[tensorboard_callback, reduce_lr]
)

Epoch 1/20


C:\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


241/259 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.4454 - loss: 1.1927

C:\Python\Python312\Lib\site-packages\PIL\Image.py:3368: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


259/259 ━━━━━━━━━━━━━━━━━━━━ 418s 2s/step - accuracy: 0.4468 - loss: 1.1893 - val_accuracy: 0.5374 - val_loss: 1.0716 - learning_rate: 0.0010
Epoch 2/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 359s 1s/step - accuracy: 0.4912 - loss: 1.1321 - val_accuracy: 0.5616 - val_loss: 1.0352 - learning_rate: 0.0010
Epoch 3/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 407s 2s/step - accuracy: 0.5649 - loss: 1.0566 - val_accuracy: 0.6263 - val_loss: 1.0168 - learning_rate: 0.0010
Epoch 4/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 401s 2s/step - accuracy: 0.5990 - loss: 1.0323 - val_accuracy: 0.6359 - val_loss: 0.9543 - learning_rate: 0.0010
Epoch 5/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 405s 2s/step - accuracy: 0.6238 - loss: 1.0052 - val_accuracy: 0.6610 - val_loss: 0.9104 - learning_rate: 0.0010
Epoch 6/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 363s 1s/step - accuracy: 0.6545 - loss: 0.9655 - val_accuracy: 0.7103 - val_loss: 0.8408 - learning_rate: 0.0010
Epoch 7/20
259/259 ━━━━━━━━━━━━━━━━━━━━ 324s 1s/step - accuracy: 0.6792 - loss: 0.9100 - val_

In [11]:
# Evaluar el modelo
loss, accuracy = model.evaluate(validation_generator)
print(f'Accuracy: {accuracy}')

65/65 ━━━━━━━━━━━━━━━━━━━━ 58s 894ms/step - accuracy: 0.7666 - loss: 0.7179
Accuracy: 0.7783679366111755


In [12]:
# Guardar el modelo en un archivo .h5
model.save("modelo_mosquitos.h5")
